In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import confusion_matrix

In [ ]:
# categories

GAME_CATEGORIES = ['DILETTAN', 'ASTROPE', 'GLENDALE', 'GRANGER', 'CAVCADE']
REAL_CATEGORIES = ['2009 Toyota Prius', '2015 BMW M3', '1992 Mercedes-Benz 500E',
                   '2016 Dodge Ram Rebel', '2016 Audi Q7']

In [ ]:
# pipeline parameters

IMG_SIZE = 50
BATCH_SIZE = 32
EPOCHS = 40
IMAGES_PER_CATEGORY = 90

In [ ]:
REBUILD_DATA = False

class GTACars():
    _img_size = IMG_SIZE
    _categories = GAME_CATEGORIES
    _training_data = []
    
    def make_training_data(self):
        self.labels = {}
        for i, label in enumerate(self._categories):
            self.labels[label] = i
        for label in self.labels:
            label_count = 0
            for f in os.listdir(f'processed_images/{label}'):
                try:
                    path = os.path.join(f'processed_images/{label}', f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self._img_size, self._img_size))
                    self._training_data.append([np.array(img), self.labels[label]])
                    label_count += 1
                    if label_count == IMAGES_PER_CATEGORY:
                        break
                except Exception as e:
                    pass
        np.random.shuffle(self._training_data)
        np.save('training_data.npy', self._training_data)

if REBUILD_DATA:
    gta_cars = GTACars()
    gta_cars.make_training_data()

In [ ]:
# loading training data

training_data = np.load('training_data.npy', allow_pickle=True)
random.shuffle(training_data)
print(f'Loaded {len(training_data)} training images.')

In [ ]:
# displaying example of training data

print(f'Training Data Example Class Number: {training_data[0][1]}')
print(f'Training Data Example Category:     {REAL_CATEGORIES[training_data[0][1]]}')

plt.imshow(training_data[0][0], cmap='gray')
plt.show()

In [ ]:
# network architecture

class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        
        x = torch.randn(IMG_SIZE, IMG_SIZE).view(-1, 1, IMG_SIZE, IMG_SIZE)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 5)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

net = Net()

In [ ]:
# optimizer and loss function

optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
# formatting image and label tensors

X = torch.Tensor([i[0] for i in training_data]).view(-1, IMG_SIZE, IMG_SIZE)
X = X / 255.0
y = torch.Tensor([i[1] for i in training_data]).type(torch.LongTensor)

In [ ]:
# splitting data into train and test

TEST_RATIO = 0.1
test_size = int(len(X) * TEST_RATIO)

train_X = X[:-test_size]
train_y = y[:-test_size]

test_X = X[-test_size:]
test_y = y[-test_size:]

print(f'Training Sample Count: {len(train_X)}')
print(f'Test Sample Count:     {len(test_X)}')

In [ ]:
# training loop

n_iter = 0
epoch_losses = []
for epoch in range(EPOCHS):
    total_loss = 0
    for i in range(0, len(train_X), BATCH_SIZE):
        batch_X = train_X[i:i + BATCH_SIZE].view(-1, 1, IMG_SIZE, IMG_SIZE)
        batch_y = train_y[i:i + BATCH_SIZE]
        
        net.zero_grad()
        outputs = net(batch_X)
        loss = criterion(outputs, batch_y)
        n_iter += 1
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    epoch_losses.append(total_loss / (len(train_X) // BATCH_SIZE))
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch + 1} | Average Loss: {total_loss / (len(train_X) // BATCH_SIZE)}')

In [ ]:
plt.plot(epoch_losses)

In [ ]:
# accuracy measurment

predictions = []

correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_X)):
        real_class = test_y[i]
        net_out = net(test_X[i].view(-1, 1, IMG_SIZE, IMG_SIZE))[0]
        predicted_class = torch.argmax(net_out)
        predictions.append(predicted_class)
        if predicted_class == real_class:
            correct += 1
        total += 1

print(f'Accuracy: {round(correct / total, 3)}\n')
print(f'Confusion Matrix: \n{confusion_matrix(test_y, predictions)}')

In [ ]:
# examples of predictions with images displayed

for i in range(4):
    plt.figure()
    plt.title(f'Predicted: {REAL_CATEGORIES[predictions[i]]}\nReal: {REAL_CATEGORIES[test_y[i]]}')
    image = cv2.cvtColor(test_X[i].view(IMG_SIZE, IMG_SIZE).numpy(), cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.show()